In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from scipy.stats import zscore
from scipy import stats
from sklearn.feature_selection import VarianceThreshold
import warnings
warnings.filterwarnings('ignore')
import io
import os
import subprocess
from IPython.display import Image, display
import __main__ as main
import shutil

# Configuración de visualización
%matplotlib inline
sns.set(style="whitegrid")


### Cargamos el DATASET

### EDA

In [ ]:
# Configuración de visualización
plt.rcParams['figure.figsize'] = (16, 14)
plt.rcParams['font.size'] = 12

def load_data(file_path):
    """Carga el dataset desde un archivo CSV."""
    try:
        data = pd.read_csv(file_path)
        print("Dataset cargado exitosamente.")
        return data
    except FileNotFoundError:
        print("El archivo no existe. Verifica la ruta del archivo.")
        return None

def display_info(data):
    """Muestra la información general del dataset."""
    print("\nInformación general del dataset:")
    buffer = io.StringIO()
    data.info(buf=buffer)
    info_str = buffer.getvalue()
    print(info_str)

def display_descriptive_stats(data):
    """Muestra las estadísticas descriptivas del dataset."""
    print("\nEstadísticas descriptivas del dataset:")
    display(data.describe(include='all').transpose())

def check_and_handle_missing_values(data, strategy='mean'):
    """Verifica y maneja valores faltantes en el dataset."""
    missing_values = data.isnull().sum()
    if missing_values.any():
        print("El dataset tiene valores faltantes:")
        display(missing_values)
        
        plt.figure(figsize=(12, 6))
        missing_values.plot(kind='bar')
        plt.xlabel('Variables')
        plt.ylabel('Número de valores faltantes')
        plt.title('Valores faltantes por variable')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
        
        imputer = SimpleImputer(strategy=strategy)
        data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)
        print(f"Valores faltantes tratados con estrategia: {strategy}")
        return data_imputed
    else:
        print("El dataset no tiene valores faltantes.")
        return data

def check_data_balance(data, target_column):
    """Verifica si el dataset está balanceado con respecto a la variable objetivo."""
    if target_column not in data.columns:
        print("La variable objetivo no existe en el dataset.")
        return

    class_counts = data[target_column].value_counts()
    print("Distribución de clases en la variable objetivo:")
    display(class_counts)

    plt.figure(figsize=(10, 6))
    sns.countplot(x=target_column, data=data)
    plt.xlabel(target_column)
    plt.ylabel('Frecuencia')
    plt.title('Distribución de clases en la variable objetivo')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    if len(class_counts) > 1:
        max_count = class_counts.max()
        min_count = class_counts.min()
        imbalance_ratio = max_count / min_count
        if imbalance_ratio > 2:
            print(f"El dataset está desbalanceado. La clase mayoritaria tiene {imbalance_ratio:.2f} veces más muestras que la clase minoritaria.")
        else:
            print("El dataset está relativamente balanceado.")
    else:
        print("La variable objetivo tiene una sola clase. No se puede evaluar el balance.")

def plot_variable_distribution(data, variable):
    """Grafica la distribución de una variable numérica."""
    plt.figure(figsize=(12, 6))
    sns.histplot(data=data, x=variable, kde=True)
    plt.title(f"Distribución de {variable}")
    plt.xlabel(variable)
    plt.ylabel("Frecuencia")
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(12, 6))
    sns.boxplot(x=variable, data=data)
    plt.title(f"Boxplot de {variable}")
    plt.tight_layout()
    plt.show()

def plot_categorical_variable(data, variable):
    """Grafica la distribución de una variable categórica."""
    plt.figure(figsize=(12, 6))
    category_counts = data[variable].value_counts()
    sns.barplot(x=category_counts.index, y=category_counts.values)
    plt.title(f"Distribución de {variable}")
    plt.xlabel(variable)
    plt.ylabel("Frecuencia")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

def plot_correlation_matrix(data):
    """Grafica la matriz de correlación de las variables numéricas."""
    numeric_columns = data.select_dtypes(include=[np.number]).columns
    if len(numeric_columns) == 0:
        print("No hay variables numéricas en el dataset.")
        return

    corr_matrix = data[numeric_columns].corr()
    plt.figure(figsize=(16, 14))
    sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", linewidths=0.5, fmt=".2f", annot_kws={"fontsize": 8}, square=True, cbar_kws={"shrink": 0.8})
    plt.title("Matriz de Correlación")
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    print("Matriz de correlación generada.")

def detect_outliers(data, threshold=3):
    """Detecta outliers en el dataset utilizando el método de Z-score."""
    numeric_columns = data.select_dtypes(include=[np.number]).columns
    outliers = {}
    
    for col in numeric_columns:
        col_zscore = zscore(data[col])
        outliers[col] = data[(np.abs(col_zscore) > threshold)]
    
    print(f"Outliers detectados (umbral de Z-score: {threshold}):")
    for col, outlier_data in outliers.items():
        if not outlier_data.empty:
            print(f"\nColumna: {col}")
            display(outlier_data)
        else:
            print(f"\nColumna: {col} - Sin outliers detectados")

def plot_pairplot(data, target_column=None):
    """Genera un pairplot de las variables numéricas."""
    numeric_columns = data.select_dtypes(include=[np.number]).columns
    plt.figure(figsize=(20, 20))
    sns.pairplot(data[numeric_columns], hue=target_column if target_column in data.columns else None)
    plt.tight_layout()
    plt.show()
    
def analyze_categorical_cardinality(data):
    """Analiza la cardinalidad de las variables categóricas."""
    cat_columns = data.select_dtypes(include=['object', 'category']).columns
    cardinality = {col: data[col].nunique() for col in cat_columns}
    print("Cardinalidad de variables categóricas:")
    for col, card in sorted(cardinality.items(), key=lambda x: x[1], reverse=True):
        print(f"{col}: {card}")
    
    plt.figure(figsize=(12, 6))
    plt.bar(cardinality.keys(), cardinality.values())
    plt.title("Cardinalidad de variables categóricas")
    plt.xlabel("Variables")
    plt.ylabel("Cardinalidad")
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

def detect_low_variance(data, threshold=0.01):
    """Detecta variables numéricas con baja varianza."""
    num_columns = data.select_dtypes(include=[np.number]).columns
    selector = VarianceThreshold(threshold)
    selector.fit(data[num_columns])
    low_variance_features = num_columns[~selector.get_support()].tolist()
    print(f"Variables con varianza menor a {threshold}:")
    print(low_variance_features)

def analyze_skewness_kurtosis(data):
    """Analiza la asimetría y curtosis de las variables numéricas."""
    num_columns = data.select_dtypes(include=[np.number]).columns
    skewness = data[num_columns].apply(stats.skew)
    kurtosis = data[num_columns].apply(stats.kurtosis)
    
    print("Asimetría y curtosis de variables numéricas:")
    for col in num_columns:
        print(f"{col}:")
        print(f"  Asimetría: {skewness[col]:.2f}")
        print(f"  Curtosis: {kurtosis[col]:.2f}")
    
    plt.figure(figsize=(12, 6))
    plt.scatter(skewness, kurtosis)
    for i, col in enumerate(num_columns):
        plt.annotate(col, (skewness[i], kurtosis[i]))
    plt.title("Asimetría vs Curtosis")
    plt.xlabel("Asimetría")
    plt.ylabel("Curtosis")
    plt.axhline(y=0, color='r', linestyle='--')
    plt.axvline(x=0, color='r', linestyle='--')
    plt.tight_layout()
    plt.show()

def check_multicollinearity(data, threshold=0.8):
    """Identifica posibles problemas de multicolinealidad."""
    num_columns = data.select_dtypes(include=[np.number]).columns
    corr_matrix = data[num_columns].corr().abs()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    high_corr = [(col1, col2) for col1 in upper_tri.columns for col2 in upper_tri.index if upper_tri.loc[col2, col1] > threshold]
    
    print(f"Pares de variables con correlación mayor a {threshold}:")
    for col1, col2 in high_corr:
        print(f"{col1} - {col2}: {corr_matrix.loc[col1, col2]:.2f}")

def analyze_target_relationship(data, target_column):
    """Analiza la relación entre variables y la variable objetivo."""
    if target_column not in data.columns:
        print("La variable objetivo especificada no existe en el dataset.")
        return
    
    num_columns = data.select_dtypes(include=[np.number]).columns
    num_columns = num_columns.drop(target_column) if target_column in num_columns else num_columns
    
    for col in num_columns:
        plt.figure(figsize=(10, 6))
        sns.scatterplot(x=col, y=target_column, data=data)
        plt.title(f"Relación entre {col} y {target_column}")
        plt.tight_layout()
        plt.show()
    
    cat_columns = data.select_dtypes(include=['object', 'category']).columns
    for col in cat_columns:
        plt.figure(figsize=(10, 6))
        sns.boxplot(x=col, y=target_column, data=data)
        plt.title(f"Relación entre {col} y {target_column}")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

def suggest_transformations(data):
    """Sugiere posibles transformaciones para variables numéricas."""
    num_columns = data.select_dtypes(include=[np.number]).columns
    for col in num_columns:
        skewness = stats.skew(data[col])
        if abs(skewness) > 1:
            print(f"La variable {col} tiene una asimetría de {skewness:.2f}.")
            if skewness > 0:
                print("  Sugerencia: Considerar una transformación logarítmica o raíz cuadrada.")
            else:
                print("  Sugerencia: Considerar una transformación exponencial o cuadrática.")

def detect_duplicates(data):
    """Detecta filas duplicadas en el dataset."""
    duplicates = data.duplicated()
    if duplicates.any():
        print(f"Se encontraron {duplicates.sum()} filas duplicadas.")
        print("Primeras 5 filas duplicadas:")
        display(data[duplicates].head())
    else:
        print("No se encontraron filas duplicadas.")

def analyze_time_series(data, date_column):
    """Realiza un análisis básico de series temporales si aplica."""
    if date_column not in data.columns:
        print("La columna de fecha especificada no existe en el dataset.")
        return
    
    data[date_column] = pd.to_datetime(data[date_column])
    data = data.sort_values(date_column)
    
    plt.figure(figsize=(12, 6))
    for col in data.select_dtypes(include=[np.number]).columns:
        plt.plot(data[date_column], data[col], label=col)
    plt.title("Series temporales de variables numéricas")
    plt.xlabel("Fecha")
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def generate_summary_report(data, target_column=None):
    """Genera un informe resumen del análisis exploratorio."""
    report = []
    report.append(f"Resumen del Análisis Exploratorio de Datos")
    report.append(f"Tamaño del dataset: {data.shape[0]} filas, {data.shape[1]} columnas")
    report.append(f"Tipos de datos:\n{data.dtypes.value_counts()}")
    report.append(f"Valores faltantes: {data.isnull().sum().sum()}")
    
    if target_column:
        if data[target_column].dtype == 'object':
            report.append(f"Variable objetivo: {target_column} (categórica)")
            report.append(f"Clases: {', '.join(data[target_column].unique())}")
        else:
            report.append(f"Variable objetivo: {target_column} (numérica)")
            report.append(f"Rango: {data[target_column].min()} - {data[target_column].max()}")
    
    report.append("Principales hallazgos:")
    report.append("- [Añadir hallazgos importantes aquí]")
    
    report.append("Próximos pasos sugeridos:")
    report.append("- [Añadir sugerencias para el siguiente paso del análisis]")
    
    print("\n".join(report))
    
def convert_notebook_to_markdown(notebook_path):
    """
    Convierte el notebook actual a un archivo Markdown usando nbconvert.
    
    :param notebook_path: Ruta al notebook de Jupyter
    """
    try:
        result = subprocess.run(['jupyter', 'nbconvert', '--to', 'markdown', '--no-input', notebook_path], 
                                capture_output=True, text=True, check=True)
        print("Conversión exitosa:")
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print("Error durante la conversión:")
        print(e.stderr)


def main(file_path):
    try:
        # 1. Cargar los datos
        data = load_data(file_path)
        if data is None:
            print("No se puede continuar sin datos.")
            return

        # 2. Mostrar información general y estadísticas descriptivas
        display_info(data)
        display_descriptive_stats(data)

        # 3. Verificar y manejar valores faltantes
        data = check_and_handle_missing_values(data)

        # 4. Verificar balance de clases (si hay variable objetivo)
        target_column = input("Ingresa el nombre de la variable objetivo (o presiona Enter si no hay variable objetivo): ")
        if target_column.strip():
            check_data_balance(data, target_column)
        else:
            print("No se especificó variable objetivo. No se verificará el balance de clases.")

        # 5. Analizar distribuciones de variables
        categorical_columns = data.select_dtypes(include=['object', 'category']).columns
        numeric_columns = data.select_dtypes(include=[np.number]).columns

        print(f"\nNúmero de variables categóricas: {len(categorical_columns)}")
        print(f"Número de variables numéricas: {len(numeric_columns)}")

        print("\nDistribuciones de las variables categóricas:")
        for column in categorical_columns:
            plot_categorical_variable(data, column)

        print("\nDistribuciones de las variables numéricas:")
        for column in numeric_columns:
            plot_variable_distribution(data, column)

        # 6. Analizar correlaciones
        plot_correlation_matrix(data)

        # 7. Detectar outliers
        detect_outliers(data)

        # 8. Generar pairplot
        plot_pairplot(data, target_column if target_column.strip() else None)
        
        analyze_categorical_cardinality(data)
        detect_low_variance(data)
        analyze_skewness_kurtosis(data)
        check_multicollinearity(data)
        
        if target_column.strip():
            analyze_target_relationship(data, target_column)
        
        suggest_transformations(data)
        detect_duplicates(data)
        
        date_column = input("Ingresa el nombre de la columna de fecha (o presiona Enter si no hay): ")
        if date_column.strip():
            analyze_time_series(data, date_column)

    except KeyboardInterrupt:
        print("\nProceso interrumpido por el usuario. Generando informe con los datos disponibles.")
    finally:
        # Generar el informe Markdown
        generate_summary_report(data, target_column if target_column.strip() else None)
        convert_notebook_to_markdown('diabetes.ipynb')

    print("Proceso completado.")



In [ ]:
if __name__ == "__main__":
    file_path = "diabetes.csv"  # Asegúrate de que esta ruta sea correcta
    main(file_path)
    
    # Convierte el notebook a Markdown
    notebook_path = os.path.abspath("diabetes.ipynb")
    convert_notebook_to_markdown(notebook_path)

In [ ]:
!jupyter nbconvert --to markdown --no-input diabetes.ipynb

## Machine learning, preparacion para los modelos.

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from sklearn.feature_selection import mutual_info_classif
import matplotlib.pyplot as plt
import seaborn as sns
from prettytable import PrettyTable

# Función para guardar figuras como imágenes
def save_figure(fig, filename):
    fig.savefig(filename)
    plt.close(fig)

# Función para realizar el tuning de hiperparámetros
def tune_model(model, params, X_train, y_train):
    grid_search = RandomizedSearchCV(model, params, cv=5, scoring='roc_auc', n_iter=20, random_state=42)
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_, grid_search.best_score_

# Función para evaluar el modelo
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    report = classification_report(y_test, y_pred, output_dict=True)
    cm = confusion_matrix(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    # Curva Precision-Recall
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    avg_precision = average_precision_score(y_test, y_pred_proba)
    
    return report, cm, roc_auc, (precision, recall, avg_precision)

# Iniciar el contenido del archivo Markdown
md_content = "# Resultados del Aprendizaje Automático para la Predicción de Diabetes\n\n"

# 1. Cargar y preparar los datos
data = pd.read_csv("diabetes.csv")
X = data.drop("Outcome", axis=1)
y = data["Outcome"]

# 2. Manejo de valores atípicos (ejemplo simple)
def remove_outliers(df):
    for column in df.columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

X = remove_outliers(X)
y = y[X.index]

# 3. Imputación de valores faltantes
imputer = SimpleImputer(strategy='median')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# 4. Normalización
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# 5. Selección de características
mi_scores = mutual_info_classif(X_scaled, y)
mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)
md_content += "## Importancia de características (Información Mutua)\n\n"
md_content += "| Característica | Puntuación MI |\n|----------------|---------------|\n"
for feature, score in mi_scores.items():
    md_content += f"| {feature} | {score:.4f} |\n"
md_content += "\n"

# 6. División de datos
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

# 7. Definición de modelos
models = {
    "Logistic Regression": {
        "model": LogisticRegression(),
        "params": {
            "C": [0.001, 0.01, 0.1, 1, 10, 100],
            "penalty": ["l1", "l2"],
            "solver": ["liblinear", "saga"]
        }
    },
    "Decision Tree": {
        "model": DecisionTreeClassifier(),
        "params": {
            "max_depth": [3, 5, 7, 10, None],
            "min_samples_split": [2, 5, 10],
            "min_samples_leaf": [1, 2, 4]
        }
    },
    "Random Forest": {
        "model": RandomForestClassifier(),
        "params": {
            "n_estimators": [100, 200, 300],
            "max_depth": [3, 5, 7, 10, None],
            "min_samples_split": [2, 5, 10],
            "min_samples_leaf": [1, 2, 4]
        }
    },
    "SVM": {
        "model": SVC(probability=True),
        "params": {
            "C": [0.1, 1, 10],
            "kernel": ["rbf", "poly"],
            "gamma": ["scale", "auto", 0.1, 1]
        }
    },
    "XGBoost": {
        "model": XGBClassifier(),
        "params": {
            "n_estimators": [100, 200, 300],
            "max_depth": [3, 5, 7],
            "learning_rate": [0.01, 0.1, 0.3],
            "subsample": [0.8, 0.9, 1.0]
        }
    },
    "Gradient Boosting": {
        "model": GradientBoostingClassifier(),
        "params": {
            "n_estimators": [100, 200, 300],
            "max_depth": [3, 5, 7],
            "learning_rate": [0.01, 0.1, 0.3],
            "subsample": [0.8, 0.9, 1.0]
        }
    }
}

# 8. Entrenamiento, tuning y evaluación de modelos
results = []
md_content += "## Resultados de los Modelos\n\n"
md_content += "| Modelo | ROC AUC | Precisión | Recall | F1-Score |\n"
md_content += "|--------|---------|-----------|--------|----------|\n"

for name, model_info in models.items():
    print(f"Tuning y evaluando {name}...")
    
    # Tuning de hiperparámetros
    grid_search = RandomizedSearchCV(model_info["model"], model_info["params"], cv=5, scoring='roc_auc', n_iter=20, random_state=42)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    
    # Evaluación del modelo
    y_pred = best_model.predict(X_test)
    y_pred_proba = best_model.predict_proba(X_test)[:, 1]
    
    report = classification_report(y_test, y_pred, output_dict=True)
    cm = confusion_matrix(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    results.append({
        "Modelo": name,
        "ROC AUC": roc_auc,
        "Precisión": report['weighted avg']['precision'],
        "Recall": report['weighted avg']['recall'],
        "F1-Score": report['weighted avg']['f1-score']
    })
    
    md_content += f"| {name} | {roc_auc:.4f} | {report['weighted avg']['precision']:.4f} | {report['weighted avg']['recall']:.4f} | {report['weighted avg']['f1-score']:.4f} |\n"
    
    # Agregar resultados detallados al contenido Markdown
    md_content += f"\n## Evaluación Detallada de {name}\n\n"
    md_content += "### Informe de Clasificación\n\n"
    md_content += f"```\n{classification_report(y_test, y_pred)}\n```\n\n"
    
    # Matriz de Confusión
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
    ax.set_title(f'Matriz de Confusión - {name}')
    ax.set_ylabel('Verdadero')
    ax.set_xlabel('Predicho')
    save_figure(fig, f"confusion_matrix_{name.replace(' ', '_')}.png")
    md_content += f"### Matriz de Confusión\n\n![Matriz de Confusión - {name}](confusion_matrix_{name.replace(' ', '_')}.png)\n\n"
    
    # Curva ROC
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})')
    ax.plot([0, 1], [0, 1], linestyle='--')
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title(f'ROC Curve - {name}')
    ax.legend()
    save_figure(fig, f"roc_curve_{name.replace(' ', '_')}.png")
    md_content += f"### Curva ROC\n\n![Curva ROC - {name}](roc_curve_{name.replace(' ', '_')}.png)\n\n"
    
    # Curva Precision-Recall
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    avg_precision = average_precision_score(y_test, y_pred_proba)
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(recall, precision, label=f'PR Curve (AP = {avg_precision:.2f})')
    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')
    ax.set_title(f'Precision-Recall Curve - {name}')
    ax.legend()
    save_figure(fig, f"pr_curve_{name.replace(' ', '_')}.png")
    md_content += f"### Curva Precision-Recall\n\n![Curva PR - {name}](pr_curve_{name.replace(' ', '_')}.png)\n\n"

# Encontrar el mejor modelo basado en ROC AUC
best_model_info = max(results, key=lambda x: x['ROC AUC'])
md_content += f"\n## Mejor Modelo: {best_model_info['Modelo']}\n\n"
md_content += f"ROC AUC: {best_model_info['ROC AUC']:.4f}\n"
md_content += f"Precisión: {best_model_info['Precisión']:.4f}\n"
md_content += f"Recall: {best_model_info['Recall']:.4f}\n"
md_content += f"F1-Score: {best_model_info['F1-Score']:.4f}\n\n"

# Importancia de características para el mejor modelo (si es aplicable)
best_model = models[best_model_info['Modelo']]['model']
if hasattr(best_model, 'feature_importances_'):
    importances = best_model.feature_importances_
    feature_importance = pd.DataFrame({'feature': X.columns, 'importance': importances})
    feature_importance = feature_importance.sort_values('importance', ascending=False)

    fig, ax = plt.subplots(figsize=(10, 6))
    sns.barplot(x='importance', y='feature', data=feature_importance, ax=ax)
    ax.set_title(f'Importancia de características ({best_model_info["Modelo"]})')
    save_figure(fig, "feature_importance.png")
    md_content += "## Importancia de Características\n\n"
    md_content += "![Importancia de Características](feature_importance.png)\n\n"

# Guardar el contenido en un archivo Markdown
with open("ML_results_extended.md", "w") as f:
    f.write(md_content)

print("Los resultados extendidos del aprendizaje automático se han guardado en 'ML_results_extended.md'")

Mejores parámetros para Random Forest: {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 200}
Los resultados del aprendizaje automático se han guardado en 'ML_results.md'


## Elección de Modelos y Métricas

Para este problema de clasificación binaria (predicción de diabetes), hemos seleccionado los siguientes modelos:

1. **Regresión Logística**: Un modelo lineal simple que sirve como línea base para comparar con modelos más complejos.

2. **Árbol de Decisión**: Un modelo no lineal que puede capturar interacciones complejas entre características.

3. **Random Forest**: Un conjunto de árboles de decisión que generalmente mejora la precisión y reduce el sobreajuste.

4. **SVM (Support Vector Machine)**: Eficaz en espacios de alta dimensionalidad y cuando hay una clara separación entre clases.

5. **XGBoost**: Un algoritmo de gradient boosting conocido por su alto rendimiento en una variedad de problemas de aprendizaje automático.

Estos modelos fueron elegidos por su diversidad en enfoques y su eficacia probada en problemas de clasificación similares.

### Métricas de Evaluación

Utilizamos las siguientes métricas para evaluar el rendimiento de nuestros modelos:

1. **ROC AUC (Area Under the Receiver Operating Characteristic Curve)**: Mide la capacidad del modelo para distinguir entre clases. Un valor más alto indica un mejor rendimiento.

2. **Precisión**: La proporción de predicciones positivas correctas entre todas las predicciones positivas.

3. **Recall (Sensibilidad)**: La proporción de predicciones positivas correctas entre todos los casos positivos reales.

4. **F1-Score**: La media armónica de precisión y recall, proporcionando un balance entre ambas métricas.

5. **Matriz de Confusión**: Nos permite visualizar los verdaderos positivos, falsos positivos, verdaderos negativos y falsos negativos.

Estas métricas nos ayudan a evaluar el rendimiento general del modelo, su capacidad para identificar correctamente los casos positivos (pacientes con diabetes) y su tasa de falsos positivos y negativos.

### Resultados Iniciales

Aquí se muestra una tabla con los resultados iniciales de la validación cruzada para cada modelo:

| Modelo | ROC AUC Media | Desviación Estándar |
|--------|---------------|---------------------|
| Logistic Regression | 0.XXX | 0.XXX |
| Decision Tree | 0.XXX | 0.XXX |
| Random Forest | 0.XXX | 0.XXX |
| SVM | 0.XXX | 0.XXX |
| XGBoost | 0.XXX | 0.XXX |

Basándonos en estos resultados, procedimos a ajustar los hiperparámetros del modelo con mejor rendimiento (en este caso, Random Forest) para optimizar aún más su rendimiento.